In [3]:
from datetime import datetime
from io import DEFAULT_BUFFER_SIZE
import pandas as pd
import openpyxl
from sqlalchemy import create_engine

sourceFile="Family expenses.xlsm"
outputFile="Family expenses.csv"
outputJSON="Family expenses.json"

book = openpyxl.load_workbook(
    sourceFile, data_only=True, read_only=True
)

dfExpense = pd.read_excel(
    sourceFile,
    sheet_name="Expenses DB",
    skiprows=9,
    usecols="J,Q:U",
)
dfIncome = pd.read_excel(
    sourceFile,
    sheet_name="Income DB",
    skiprows=9,
    usecols="J,Q:U",
)

dfExpense["Amount"] *= -1
dfIncome.rename(columns={"Type ": "Category"}, inplace=True)
dfIncome["Store"] = "NA"
dfIncome["Exclude in WE"] = "NA"
dfResult = dfExpense.append(dfIncome)
dfResult.sort_values(by="Date", inplace=True)
dfResult.rename(columns={"Exclude in WE": "ExcludeWE"}, inplace=True)
dfResult["Date"] = dfResult["Date"].dt.strftime("%Y-%m-%dT00:00:00.000Z")
dfResult["created_at"] = datetime.now().strftime("%Y-%m-%dT00:00:00.000Z")
dfResult["updated_at"] = datetime.now().strftime("%Y-%m-%dT00:00:00.000Z")
cols = [
    "Date",
    "Store",
    "Category",
    "Amount",
    "ExcludeWE",
    "Description",
]
dfResult = dfResult[cols]

In [28]:
dfResult.reset_index(drop=True)

,Date,created_at,updated_at,Store,Category,Amount,ExcludeWE,Description
0,2014-01-10T00:00:00.000Z,2021-07-20T00:00:00.000Z,2021-07-20T00:00:00.000Z,NA,Savings,37402.26,NA,Savings
1,2014-01-11T00:00:00.000Z,2021-07-20T00:00:00.000Z,2021-07-20T00:00:00.000Z,Aldi,Daily Expenses,-4.76,NO,"Milk, Water, Tuna"
2,2014-01-12T00:00:00.000Z,2021-07-20T00:00:00.000Z,2021-07-20T00:00:00.000Z,Aldi,Daily Expenses,-16.00,NO,"Wine, juice, cream cheese, corn, eggs and wraps"
3,2014-01-12T00:00:00.000Z,2021-07-20T00:00:00.000Z,2021-07-20T00:00:00.000Z,Victoria Market,Daily Expenses,-13.00,NO,"Vegetables, meet, lentins and fruits"
4,2014-01-13T00:00:00.000Z,2021-07-20T00:00:00.000Z,2021-07-20T00:00:00.000Z,eBay,Gift,-98.90,YES,Bathrobe gift for Russia
...,...,...,...,...,...,...,...,...
8135,2021-07-18T00:00:00.000Z,2021-07-20T00:00:00.000Z,2021-07-20T00:00:00.000Z,Others,Gift,-71.00,YES,sheets for dani and tiffani
8136,2021-07-18T00:00:00.000Z,2021-07-20T00:00:00.000Z,2021-07-20T00:00:00.000Z,Coles,Daily Expenses,-13.59,NO,food
8137,2021-07-18T00:00:00.000Z,2021-07-20T00:00:00.000Z,2021-07-20T00:00:00.000Z,Others,Education,-20.95,YES,dance class
8138,2021-07-18T00:00:00.000Z,2021-07-20T00:00:00.000Z,2021-07-20T00:00:00.000Z,Liquorland,Alcohol,-12.00,YES,beer
